In [1]:
import sys
import pandas as pd
from sklearn.model_selection import GroupKFold
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython import display
from glob import glob
import cv2
from tqdm.auto import tqdm
import pydicom
import h5py

In [2]:
train_descriptions = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv")
train_coord_df = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv")
train_coord_df = train_coord_df.merge(train_descriptions, on=["series_id", "study_id"], how="left")
train_df = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/train.csv")


train_descriptions = pd.read_csv(
    "rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv"
)
train_coord_df = pd.read_csv(
    "rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv"
)
train_coord_df = train_coord_df.merge(
    train_descriptions, on=["series_id", "study_id"], how="left"
)
train_df = pd.read_csv(
    "rsna-2024-lumbar-spine-degenerative-classification/train.csv"
)
train_df["fold_id"] = -1
for i, (train_index, valid_index) in enumerate(
    GroupKFold(n_splits=5).split(
        train_df, np.arange(len(train_df)), train_df.study_id
    )
):
    train_df.loc[valid_index, "fold_id"] = i
train_coord_df = train_coord_df.merge(
    train_df.loc[:, ["study_id", "fold_id"]], on=["study_id"], how="left"
)
train_coord_df = train_coord_df.sort_values(
    by=["study_id", "series_id", "level", "condition"]
).reset_index(drop=True)
coord_label_num = (
    train_coord_df.groupby("series_id")
    .count()
    .sort_values("series_id")
    .study_id.to_numpy()
)
SCS = (
    train_coord_df.groupby("series_id")
    .head(1)
    .sort_values("series_id")
    .condition.to_numpy()
)
series_id = (
    train_coord_df.groupby("series_id").head(1).sort_values("series_id").series_id
)
new_series_id = series_id[
    ((SCS == "Spinal Canal Stenosis") & (coord_label_num == 5))
    | ((SCS != "Spinal Canal Stenosis") & (coord_label_num == 10))
]
new_train_coord_df = (
    train_coord_df[train_coord_df.series_id.isin(new_series_id)]
    .sort_values(by=["series_id", "level"])
    .reset_index(drop=True)
)
wrong_series_ids = [
    221289021,
    1735851779,
    880361156,
    1921917205,
    2231471633,
    737753815,
    1488857550,
    3736941525,
    1490272456,
    3086719329,
    1485193299,
    3521409198,
    816381378,
]

new_train_coord_df =  new_train_coord_df[~new_train_coord_df.series_id.isin(wrong_series_ids)]


In [3]:
label = []
for idx in tqdm(range(len(new_train_coord_df))):
    study_id, condition, level = new_train_coord_df.iloc[idx][["study_id", "condition", "level"]]
    cond_target = "_".join(condition.lower().split(" ") + level.lower().split("/"))
    label.append(train_df[train_df.study_id==study_id].loc[:, cond_target].iloc[0])
new_train_coord_df["label"] = label
new_train_coord_df

  0%|          | 0/43870 [00:00<?, ?it/s]

,study_id,series_id,instance_number,condition,level,x,y,series_description,fold_id,label
0,3996069892,10996,13,Left Neural Foraminal Narrowing,L1/L2,266.827697,166.544283,Sagittal T1,0,Normal/Mild
1,3996069892,10996,6,Right Neural Foraminal Narrowing,L1/L2,264.630404,163.686737,Sagittal T1,0,Normal/Mild
2,3996069892,10996,12,Left Neural Foraminal Narrowing,L2/L3,252.811594,205.294686,Sagittal T1,0,Normal/Mild
3,3996069892,10996,6,Right Neural Foraminal Narrowing,L2/L3,258.064459,197.391921,Sagittal T1,0,Normal/Mild
4,3996069892,10996,12,Left Neural Foraminal Narrowing,L3/L4,249.513688,243.220612,Sagittal T1,0,Moderate
...,...,...,...,...,...,...,...,...,...,...
43925,916362094,4294540297,11,Spinal Canal Stenosis,L1/L2,254.122744,159.104693,Sagittal T2/STIR,2,Normal/Mild
43926,916362094,4294540297,10,Spinal Canal Stenosis,L2/L3,239.566787,209.241877,Sagittal T2/STIR,2,Moderate
43927,916362094,4294540297,10,Spinal Canal Stenosis,L3/L4,229.054152,256.144404,Sagittal T2/STIR,2,Moderate
43928,916362094,4294540297,9,Spinal Canal Stenosis,L4/L5,231.480144,300.620939,Sagittal T2/STIR,2,Severe


In [4]:
new_train_coord_df[new_train_coord_df.series_description == "Axial T2"]

,study_id,series_id,instance_number,condition,level,x,y,series_description,fold_id,label
10,2767326159,223384,6,Left Subarticular Stenosis,L1/L2,173.698384,156.122083,Axial T2,3,Normal/Mild
11,2767326159,223384,5,Right Subarticular Stenosis,L1/L2,136.549107,153.142857,Axial T2,3,Normal/Mild
12,2767326159,223384,16,Left Subarticular Stenosis,L2/L3,174.272890,150.377020,Axial T2,3,Normal/Mild
13,2767326159,223384,15,Right Subarticular Stenosis,L2/L3,139.977679,150.857143,Axial T2,3,Normal/Mild
14,2767326159,223384,24,Left Subarticular Stenosis,L3/L4,171.974865,146.929982,Axial T2,3,Moderate
...,...,...,...,...,...,...,...,...,...,...
43890,4084395132,4289954644,17,Right Subarticular Stenosis,L3/L4,246.857143,227.638484,Axial T2,3,Normal/Mild
43891,4084395132,4289954644,11,Left Subarticular Stenosis,L4/L5,288.565962,235.848397,Axial T2,3,Severe
43892,4084395132,4289954644,11,Right Subarticular Stenosis,L4/L5,247.603499,231.370262,Axial T2,3,Moderate
43893,4084395132,4289954644,7,Left Subarticular Stenosis,L5/S1,279.609694,251.521866,Axial T2,3,Normal/Mild


In [5]:
all_new_df = {}
for series_description in  ["Sagittal T1", "Sagittal T2/STIR", "Axial T2"]:
    new_df = []
    for series_id in tqdm(new_train_coord_df[new_train_coord_df.series_description == series_description].series_id.unique()):
        series_df = new_train_coord_df[new_train_coord_df.series_id == series_id].sort_values(["level", "condition"])
        dicoms = [pydicom.dcmread(glob(f"rsna-2024-lumbar-spine-degenerative-classification/train_images/**/{series_id}/{i}.dcm")[0]) for i in series_df.instance_number]
        orig_x = np.asarray([dicom.Columns for dicom in dicoms])
        orig_y = np.asarray([dicom.Rows for dicom in dicoms])
        series_df.loc[:, "x"] = series_df.loc[:, "x"] / orig_x
        series_df.loc[:, "y"] = series_df.loc[:, "y"] / orig_y
        if series_df.series_description.iloc[0] == "Sagittal T1":
            if len(series_df) != 10:
                continue
            new_series_df = series_df[series_df.condition == "Left Neural Foraminal Narrowing"].reset_index(drop=True).copy()
            new_series_df_l = series_df[series_df.condition == "Left Neural Foraminal Narrowing"].reset_index(drop=True).copy()
            new_series_df_r = series_df[series_df.condition == "Right Neural Foraminal Narrowing"].reset_index(drop=True).copy()
            new_series_df = new_series_df.astype(
                {"instance_number": np.float32, "x": np.float32, "y": np.float32}
            )
            new_series_df.loc[:, ["x", "y"]] = (
                new_series_df_l.loc[:, ["x", "y"]].to_numpy()
                + new_series_df_r.loc[:, ["x", "y"]].to_numpy()
            ) / 2
            new_series_df = new_series_df.rename(columns={"label":"label_l", "instance_number": "instance_number_l"})
            new_series_df["label_r"] = new_series_df_r.label.to_numpy()
            new_series_df["instance_number_l"] = new_series_df_l.instance_number.to_numpy() 
            new_series_df["instance_number_r"] = new_series_df_r.instance_number.to_numpy() 
            new_series_df["x_l"] = new_series_df_l.x.to_numpy()
            new_series_df["x_r"] = new_series_df_r.x.to_numpy()
            new_series_df["y_l"] = new_series_df_l.y.to_numpy()
            new_series_df["y_r"] = new_series_df_r.y.to_numpy()

        elif series_df.series_description.iloc[0] == "Axial T2":
            if len(series_df) != 10:
                continue
            new_series_df = series_df[series_df.condition == "Left Subarticular Stenosis"].reset_index(drop=True).copy()
            new_series_df_l = series_df[series_df.condition == "Left Subarticular Stenosis"].reset_index(drop=True).copy()
            new_series_df_r = series_df[series_df.condition == "Right Subarticular Stenosis"].reset_index(drop=True).copy()
            new_series_df = new_series_df.astype(
                {"instance_number": np.float32, "x": np.float32, "y": np.float32}
            )
            new_series_df.loc[:, ["x", "y"]] = (
                new_series_df_l.loc[:, ["x", "y"]].to_numpy()
                + new_series_df_r.loc[:, ["x", "y"]].to_numpy()
            ) / 2
            new_series_df = new_series_df.rename(columns={"label":"label_l", "instance_number": "instance_number_l"})
            new_series_df["label_r"] = new_series_df_r.label.to_numpy()
            new_series_df["instance_number_l"] = new_series_df_l.instance_number.to_numpy() 
            new_series_df["instance_number_r"] = new_series_df_r.instance_number.to_numpy() 
            new_series_df["x_l"] = new_series_df_l.x.to_numpy()
            new_series_df["x_r"] = new_series_df_r.x.to_numpy()
            new_series_df["y_l"] = new_series_df_l.y.to_numpy()
            new_series_df["y_r"] = new_series_df_r.y.to_numpy()
        else:
            if len(series_df) != 5:
                continue
            new_series_df = series_df.reset_index(drop=True).copy()
        new_df.append(new_series_df)
        # break
    all_new_df[series_description] = pd.concat(new_df).reset_index(drop=True)

for key in all_new_df:
    all_new_df[key].to_csv(key.replace(" ", "_").replace("/", "-") + ".csv", index=False)

  0%|          | 0/1963 [00:00<?, ?it/s]

/tmp/ipykernel_714867/3656410605.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.51900205 0.4989024  0.48798732 0.48755985 0.48783795]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  new_series_df.loc[:, ["x", "y"]] = (
/tmp/ipykernel_714867/3656410605.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.32249123 0.39324864 0.47089536 0.5557095  0.62613902]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  new_series_df.loc[:, ["x", "y"]] = (
/tmp/ipykernel_714867/3656410605.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.54724271 0.51763904 0.49700432 0.4928766  0.5427621 ]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  new_s

  0%|          | 0/1886 [00:00<?, ?it/s]

  0%|          | 0/1481 [00:00<?, ?it/s]

/tmp/ipykernel_714867/3656410605.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.48476171 0.49101651 0.48742585 0.47397048 0.47486333]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  new_series_df.loc[:, ["x", "y"]] = (
/tmp/ipykernel_714867/3656410605.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.48322647 0.47067838 0.46261381 0.43754649 0.42054373]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  new_series_df.loc[:, ["x", "y"]] = (
/tmp/ipykernel_714867/3656410605.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.49317602 0.49524923 0.50144202 0.51487003 0.51953764]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  new_s

In [6]:
import pandas as pd
df = pd.read_csv("Axial_T2.csv")
len(df.study_id.unique())

1481

In [7]:
df = pd.read_csv("Sagittal_T1.csv")
len(df.study_id.unique())


1963

In [8]:
df = pd.read_csv("Sagittal_T2-STIR.csv")
len(df.study_id.unique())

1886

In [9]:
df = pd.read_csv("Axial_T2.csv")
df

,study_id,series_id,instance_number_l,condition,level,x,y,series_description,fold_id,label_l,label_r,instance_number_r,x_l,x_r,y_l,y_r
0,2767326159,223384,6,Left Subarticular Stenosis,L1/L2,0.484762,0.483226,Axial T2,3,Normal/Mild,Normal/Mild,5,0.542807,0.426716,0.487882,0.478571
1,2767326159,223384,16,Left Subarticular Stenosis,L2/L3,0.491017,0.470678,Axial T2,3,Normal/Mild,Normal/Mild,15,0.544603,0.437430,0.469928,0.471429
2,2767326159,223384,24,Left Subarticular Stenosis,L3/L4,0.487426,0.462614,Axial T2,3,Moderate,Normal/Mild,24,0.537421,0.437430,0.459156,0.466071
3,2767326159,223384,33,Left Subarticular Stenosis,L4/L5,0.473970,0.437546,Axial T2,3,Severe,Normal/Mild,32,0.514082,0.433859,0.434022,0.441071
4,2767326159,223384,40,Left Subarticular Stenosis,L5/S1,0.474863,0.420544,Axial T2,3,Normal/Mild,Normal/Mild,41,0.514082,0.435645,0.419659,0.421429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7400,4084395132,4289954644,30,Left Subarticular Stenosis,L1/L2,0.517043,0.518950,Axial T2,3,Normal/Mild,Moderate,30,0.559232,0.474854,0.517493,0.520408
7401,4084395132,4289954644,24,Left Subarticular Stenosis,L2/L3,0.513399,0.479592,Axial T2,3,Normal/Mild,Normal/Mild,24,0.549028,0.477770,0.479592,0.479592
7402,4084395132,4289954644,18,Left Subarticular Stenosis,L3/L4,0.514857,0.449708,Axial T2,3,Normal/Mild,Normal/Mild,17,0.547570,0.482143,0.454810,0.444606
7403,4084395132,4289954644,11,Left Subarticular Stenosis,L4/L5,0.523603,0.456268,Axial T2,3,Severe,Moderate,11,0.563605,0.483601,0.460641,0.451895


: 